In [11]:
import pandas as pd
from dataAug.tools import dataAugForTestSMOTE, dataAugForTestBorderlineSMOTE
from processTools import processRealData
import torch
from ModelClassify_3 import ModelClassify
from targeTools import testThresholdFive, Accuracy
from dataAug.tools import dataAugForTestRandomOverSampler, dataAugForTestADASYN

In [12]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/T5_578.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [14]:

smote_multiple = {}
for item in range(1, 5, 2):
    G_feature, G_label = dataAugForTestSMOTE(feature_pd, labels_pd, item, 1024)
    R_feature, R_label = processRealData(feature_pd, labels_pd, feature_num=1024)

    train_feature = pd.concat([R_feature, G_feature], axis=0)
    train_label = pd.concat([R_label, G_label], axis=0)

    test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/T5_test_mutil_label_122_finally.csv")

    from torch.utils.data import TensorDataset, DataLoader
    datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
    batch_size = len(train_feature)
    shuffle = True
    dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

    datasetTest = TensorDataset(torch.tensor(test_df.iloc[:,:1024].values), torch.tensor(test_df.iloc[:,-4:].values))
    # 创建 DataLoader 对象，设置批量大小和是否打乱数据
    batch_size = len(datasetTest)
    dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

    model = ModelClassify()
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)


    import warnings
    warnings.filterwarnings("ignore")
    GMList = {}
    for epoch in range(200):
        model.train()
        total_loss = 0.0
        for idx, data in enumerate(dataloaderTrain, 0):
            inputs, labels = data
            labels = labels.float()
            inputs = inputs.float()
            out = model(inputs)
            loss = criterion(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloaderTrain)
        threshold = 0.5
        labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
        print(f"Epoch [{epoch+1}/{300}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
        GMScore1 = testThresholdFive(epoch, model, dataloaderTest)
        GMList[epoch] = GMScore1

    best_key = max(GMList, key=GMList.get)
    best_value = GMList[best_key]
    smote_multiple[item] = best_value

Epoch [1/300], Average Loss: 0.7316, ACC: 0.247276688453159
epoch:0,bestThreshold:[0.05, 0.49, 0.05, 0.05], GM:0.3934973478317261, OAA:0.0, ACC:0.27185792349726773, F1:0.42377087473869324
Epoch [2/300], Average Loss: 0.7265, ACC: 0.2567401960784311
epoch:1,bestThreshold:[0.05, 0.5, 0.05, 0.49], GM:0.4198361337184906, OAA:0.0, ACC:0.3135245901639344, F1:0.4721314311027527
Epoch [3/300], Average Loss: 0.7195, ACC: 0.2584422657952068
epoch:2,bestThreshold:[0.49, 0.48, 0.51, 0.05], GM:0.3876503109931946, OAA:0.0, ACC:0.2711748633879781, F1:0.4204920530319214
Epoch [4/300], Average Loss: 0.7073, ACC: 0.28438180827886633
epoch:3,bestThreshold:[0.05, 0.49, 0.51, 0.05], GM:0.40366131067276, OAA:0.0, ACC:0.28825136612021857, F1:0.44180360436439514
Epoch [5/300], Average Loss: 0.7059, ACC: 0.2896241830065355
epoch:4,bestThreshold:[0.47, 0.51, 0.53, 0.05], GM:0.43109291791915894, OAA:0.0, ACC:0.3504098360655739, F1:0.503825306892395
Epoch [6/300], Average Loss: 0.6995, ACC: 0.29350490196078344
ep

In [15]:
smote_multiple

{1: [tensor(0.7642),
  0.7131147540983607,
  0.760928961748634,
  tensor(0.7828),
  tensor(0.7869),
  tensor(0.7773)],
 3: [tensor(0.7680),
  0.7213114754098361,
  0.7650273224043715,
  tensor(0.7869),
  tensor(0.7869),
  tensor(0.7801)]}